In [8]:
import numpy as np
import pandas as pd
from tslearn.metrics import cdist_dtw
from sklearn.preprocessing import StandardScaler


In [9]:
STYLE_SIGNALS = [
    "car0_velocity",
    "throttle",
    "brakes",
    "wheel_position",
    "engine_rpm"
]

In [14]:
NUMERIC_FEATURES = [
    "car0_velocity",
    "throttle",
    "brakes",
#    "wheel_position",
    "engine_rpm"
]

def load_and_prepare(path):
    df = pd.read_csv(path)
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna(subset=NUMERIC_FEATURES)
    df = df.sort_values("timestamp")
    return df


def extract_feature_timeseries(df, window_size=100):
    features = []

    for start in range(0, len(df) - window_size, window_size):
        window = df.iloc[start:start + window_size]

        feat = []
        for col in NUMERIC_FEATURES:
            feat.extend([
                window[col].mean(),
                window[col].std()
            ])
        features.append(feat)

    return np.array(features)

def build_reference_profiles(driver_files):
    references = []
    labels = []

    for label, path in enumerate(driver_files):
        df = load_and_prepare(path)
        ts = extract_feature_timeseries(df)
        references.append(ts)
        labels.append(label)

    return references, labels

def classify_with_dtw(unknown_ts, references, labels):
    distances = []

    for ref in references:
        dist = cdist_dtw(
            unknown_ts[np.newaxis, :, :],
            ref[np.newaxis, :, :]
        )[0, 0]
        distances.append(dist)

    best = np.argmin(distances)
    return labels[best], distances


In [21]:
known_drivers = [
    "./recordings/recording_florian_2.csv",
    "./recordings/recording_fabian_2.csv",
    "./recordings/recording_matthias_2.csv"
]

unknown_driver = "./recordings/recording_fabian_1.csv"

# Training
references, labels = build_reference_profiles(known_drivers)

# Unknown
unknown_df = load_and_prepare(unknown_driver)
unknown_ts = extract_feature_timeseries(unknown_df)

predicted_driver, distances = classify_with_dtw(
    unknown_ts,
    references,
    labels
)

print("DTW-Distanzen:", distances)
print("Vorhergesagter Fahrer:", predicted_driver)


C:\Users\games\AppData\Local\Temp\ipykernel_12000\927666091.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,15,16,17,18,19,24,27,31,35,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\games\AppData\Local\Temp\ipykernel_12000\927666091.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,18,21,25,29,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,69,70,71,72,73,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\games\AppData\Local\Temp\ipykernel_12000\927666091.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,20,23,27,31,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,71,72,73,78) have mixed types. Specify dtype option on import or set low_memory=False.

DTW-Distanzen: [np.float64(8860.89483268931), np.float64(8728.503312668343), np.float64(8945.82573773779)]
Vorhergesagter Fahrer: 1
